In [10]:
import scipy.io
import configparser
import datetime
import gc
import json
import os
import re
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
from typing import Any, Dict
import cv2
import h5py
import numpy as np
import pandas as pd
import scipy.io
import skimage.io as skio
import typing_extensions
from dateutil import tz
from hdmf.backends.hdf5.h5_utils import H5DataIO
from hdmf.data_utils import DataChunkIterator
from ndx_multichannel_volume import CElegansSubject, OpticalChannelReferences, OpticalChannelPlus, ImagingVolume, \
    MultiChannelVolume, MultiChannelVolumeSeries, SegmentationLabels
from pynwb import NWBFile, NWBHDF5IO
from pynwb.behavior import SpatialSeries, Position, BehavioralTimeSeries, BehavioralEvents
from pynwb.ophys import ImageSegmentation, PlaneSegmentation, \
    DfOverF, RoiResponseSeries, Fluorescence
from pynwb.image import ImageSeries
from tifffile import TiffFile
from tifffile import imread
from tqdm import tqdm
import nd2reader
import nrrd
from pathlib import Path

file = '/Users/danielysprague/foco_lab/data/NWB_test/2022-06-14-01.nwb'

with NWBHDF5IO(file, mode='r') as read_io:
    nwbfile = read_io.read()
    # ...  # modify nwbfile

    nwbfile.s = CElegansSubject(
        subject_id="2022-06-14-01.nwb",
        date_of_birth=datetime(2022,6,14),
        growth_stage='YA',
        growth_stage_time=str(np.nan),
        cultivation_temp=20.,
        description='updated subject',
        species="http://purl.obolibrary.org/obo/NCBITaxon_6239",
        sex="O",
        strain='asdf'
    )

    subject = nwbfile.subject

    print(subject)
nwbfile

    #nwbfile.set_modified()  # this may be necessary if the modifications are changes to attributes

    #with NWBHDF5IO('/Users/danielysprague/foco_lab/data/NWB_test/fixed.nwb', mode='w') as export_io:
    #    export_io.export(src_io=read_io, nwbfile=nwbfile)

dict_keys(['name', 'labels', 'description', 'ImageSegmentation', 'MCVSegmentation', 'MCVSeriesSegmentation'])
subject abc.CElegansSubject at 0x11300490896
Fields:
  age__reference: birth
  cultivation_temp: 22.0
  date_of_birth: 2022-06-14 00:00:00-07:00
  description: Ventral cord at the bottom when head points to the left on confocal images (what the neuronal manual suggests)
Young adult with a few mature eggs
Some pumps here and there
Coiling for ~5 loops at t~220, coiling for ~5 loops again at t~510, coiling again for ~5 loop at t~810, coiling briefly again t ~1100, long coiling ~8 loops at t~1400, long coiling again t~1550
The second half is slightly out of focus (the edge of 3-5 neurons at the upper border in z-dimension), should not affect registration in candy’s experience

  growth_stage: young adult
  growth_stage_time: nan
  sex: O
  species: http://purl.obolibrary.org/obo/NCBITaxon_6239
  strain: SWF702
  subject_id: 2022-06-14-01-SWF702



/Applications/anaconda3/envs/NWB-dev/lib/python3.11/site-packages/pynwb/file.py:146: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['date_of_birth'] = _add_missing_timezone(date_of_birth)


root pynwb.file.NWBFile at 0x11325974288
Fields:
  acquisition: {
    NeuroPALImageRaw <class 'ndx_multichannel_volume.ndx_multichannel_volume.MultiChannelVolume'>
  }
  devices: {
    NIR Config <class 'pynwb.device.Device'>,
    Spinning Disk Confocal <class 'pynwb.device.Device'>
  }
  file_create_date: [datetime.datetime(2024, 1, 9, 1, 56, 43, 545518, tzinfo=tzoffset(None, -28800))]
  identifier: 2022-06-14-01
  imaging_planes: {
    CalciumImVol <class 'ndx_multichannel_volume.ndx_multichannel_volume.ImagingVolume'>,
    NeuroPALImVol <class 'ndx_multichannel_volume.ndx_multichannel_volume.ImagingVolume'>
  }
  institution: MIT
  lab: Flavell Laboratory
  processing: {
    Behavior <class 'pynwb.base.ProcessingModule'>,
    CalciumActivity <class 'pynwb.base.ProcessingModule'>,
    NeuroPAL <class 'pynwb.base.ProcessingModule'>
  }
  related_publications: ['https://doi.org/10.1016/j.cell.2023.07.035']
  session_description: Ventral cord at the bottom when head points to the left on confocal images (what the neuronal manual suggests)
Young adult with a few mature eggs
Some pumps here and there
Coiling for ~5 loops at t~220, coiling for ~5 loops again at t~510, coiling again for ~5 loop at t~810, coiling briefly again t ~1100, long coiling ~8 loops at t~1400, long coiling again t~1550
The second half is slightly out of focus (the edge of 3-5 neurons at the upper border in z-dimension), should not affect registration in candy’s experience

  session_start_time: 2022-06-14 00:00:00-07:00
  subject: subject abc.CElegansSubject at 0x11300490896
Fields:
  age__reference: birth
  cultivation_temp: 22.0
  date_of_birth: 2022-06-14 00:00:00-07:00
  description: Ventral cord at the bottom when head points to the left on confocal images (what the neuronal manual suggests)
Young adult with a few mature eggs
Some pumps here and there
Coiling for ~5 loops at t~220, coiling for ~5 loops again at t~510, coiling again for ~5 loop at t~810, coiling briefly again t ~1100, long coiling ~8 loops at t~1400, long coiling again t~1550
The second half is slightly out of focus (the edge of 3-5 neurons at the upper border in z-dimension), should not affect registration in candy’s experience

  growth_stage: young adult
  growth_stage_time: nan
  sex: O
  species: http://purl.obolibrary.org/obo/NCBITaxon_6239
  strain: SWF702
  subject_id: 2022-06-14-01-SWF702

  timestamps_reference_time: 2022-06-14 00:00:00-07:00